## Install Requirements
******Remember to restart the kernel after installing

In [1]:
%pip install --upgrade pip
%pip install -r ../requirements.txt

Note: you may need to restart the kernel to use updated packages.
Ignoring tensorflow: markers 'sys_platform == "linux"' don't match your environment
Note: you may need to restart the kernel to use updated packages.


In [3]:
import os, sys, shutil
import numpy as np
import matplotlib.pyplot as plt
import cv2
from sklearn.model_selection import train_test_split
import tensorflow as tf

## Helper Functions

In [4]:
def calc_median(frames):
    median_frame = np.median(frames, axis=0).astype(dtype=np.uint8)
    return median_frame

def doMovingAverageBGS(image, prev_frames):
    median_img = calc_median(prev_frames)
    image = cv2.absdiff(image, median_img)
    return image

In [5]:
def extractImages(pathIn, pathOut, leakRange, nonleakRange, currCountLeak, currCountNonLeak):

  '''
  Input:
    String: pathIn should be the path of the video 
    String: pathOut should be the path of the folder where data is being stored for testing or training
    Tuple: range of leak frames from video
    Tuple: range of nonleak frames from video

  Output:
    creates two subfolders in pathOut called Leaks and Nonleaks
      Leaks folder contains the frames where there are leaks
      Nonleaks folder contains the frames where there are noleaks
  '''

  leakPath = os.path.join(pathOut, "Leak")
  nonleakPath = os.path.join(pathOut, "Nonleaks")
  
  os.makedirs(leakPath, exist_ok=True)
  os.makedirs(nonleakPath, exist_ok=True)

  def helper(pathIn, pathOut, range, isLeak, currCountLeak, currCountNonLeak):
    '''
    Might need to clean this up, but this was extracted from the original extractImages from the previous implementation
    
    '''
    #setting up moving average list
    prev_imgs = []
    prev_limit = 210 #210 in paper

    start = range[0] * 1000 # converting seconds to milliseconds
    end = range[1] * 1000
    cap = cv2.VideoCapture(pathIn)
    cap.set(cv2.CAP_PROP_POS_MSEC, start)
    success = True

    if cap.isOpened():
      while success and start < end:  
          success, image = cap.read()
          image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

          start = cap.get(cv2.CAP_PROP_POS_MSEC)
          if success:

            prev_imgs.append(image)
            if len(prev_imgs) > prev_limit:
                prev_imgs.pop(0)
          
            processed_img = doMovingAverageBGS(image, prev_imgs) #to generalize might need to make this function as a parameter
            
            if isLeak:
                cv2.imwrite(os.path.join(pathOut, "leak.frame%d.jpg" % currCountLeak), processed_img)     # save frame as JPEG file
                currCountLeak += 1
            else:
                cv2.imwrite(os.path.join(pathOut, "nonleak.frame%d.jpg" % currCountNonLeak), processed_img)
                currCountNonLeak += 1
          else:
            break
      cap.release()
    cv2.destroyAllWindows()
    if isLeak:
       return currCountLeak
    else:
       return currCountNonLeak
  # call helper for both nonLeak and leak and get updated counts
  updated_currCountNonLeak = helper(pathIn, nonleakPath, nonleakRange, isLeak=False, currCountLeak=currCountLeak, currCountNonLeak=currCountNonLeak)
  updated_currCountLeak = helper(pathIn, leakPath, leakRange, isLeak=True,currCountLeak=currCountLeak, currCountNonLeak=currCountNonLeak)
  
  return updated_currCountNonLeak, updated_currCountLeak

## Setting Up Directories
### Make sure frame_data_dir is set to a different name

In [6]:
# get generic path to directory
dir_path = os.path.dirname(os.path.realpath("__file__"))

# get all raw video data directories
data_dir = os.path.join(dir_path, 'data')

train_data_dir = os.path.join(data_dir, 'train')
test_data_dir = os.path.join(data_dir, 'test')

frame_data_dir = os.path.join(dir_path, 'frame_data_movingAvg_Adam')
frame_train_data_dir = os.path.join(frame_data_dir, 'train')
frame_test_data_dir = os.path.join(frame_data_dir, 'test')

## Setting Up Ranges for Each Video In GasVid (Excluding 18.6m and 8.8m)

In [11]:
raw_data = np.loadtxt(os.path.join(dir_path, 'GasVid_Ranges_Seconds.csv'), skiprows=1, delimiter=',', dtype=int)

ranges = list(zip(raw_data[:, 0], raw_data[:, 1:3], raw_data[:, 3:5])) #need to upload new ranges
ranges = {ranges[i][0] : (ranges[i][1], ranges[i][2]) for i in range(len(ranges))}
len(ranges)

25

In [9]:
def read_frames_from_dir(dir_path, output_path, max_vids=None):
    cur_count = 1
    currNonLeakCount = 0
    currLeakCount = 0
    
    for file in os.listdir(dir_path):
        if max_vids and cur_count > max_vids:
            break
        vid_path = os.path.join(dir_path, file)
        vid_id = int(os.path.basename(vid_path)[4:8])
        if vid_id not in ranges.keys():
            continue

        nonleak_start = ranges[vid_id][0][0]
        nonleak_end = ranges[vid_id][0][1]
        leak_start = ranges[vid_id][1][0]
        leak_end = ranges[vid_id][1][1]

        currNonLeakCount, currLeakCount = extractImages(vid_path, output_path, (leak_start, leak_end), (nonleak_start, nonleak_end), currLeakCount, currNonLeakCount)
        print("Video", vid_id)
        print("Current NonLeak Count", currNonLeakCount)
        print("Current Leak Count", currLeakCount)

        print('Done with', cur_count, "video(s)")
        cur_count += 1
    return currNonLeakCount, currLeakCount

## Reading Frames from Data Directory and Setting Them in Frame Data Directory

In [10]:
image_dim = (240, 320)
vid_count = 15 #max =>15
test_count = 10 #max =>10

total_train_NonLeak, total_train_Leak = read_frames_from_dir(train_data_dir, frame_train_data_dir, vid_count)
print("Done with Training Data")
total_test_NonLeak, total_test_Leak = read_frames_from_dir(test_data_dir, frame_test_data_dir, test_count)
print("Done with Testing Data")
#weird bug in which it if vid_count is 1 goes on to the next one
#might need to cut in amount of samples but we will see

error: OpenCV(4.8.0) /Users/xperience/GHA-OpenCV-Python/_work/opencv-python/opencv-python/opencv/modules/imgproc/src/color.cpp:182: error: (-215:Assertion failed) !_src.empty() in function 'cvtColor'


In [12]:
total_train_NonLeak, total_train_Leak = 35779, 277114
total_test_NonLeak, total_test_Leak = 23857, 184811

## Setting Up Generators for Training, Validation, and Testing

In [13]:
from keras.preprocessing.image import ImageDataGenerator

val_split = 0.2
batch_size = 32
target_size = (240, 320)


train_datagen = ImageDataGenerator(
    # featurewise_center=True, #cant do this as need entire dataset to do it | need to figure out a way in doing this
    # featurewise_std_normalization=True,
    rescale=1. / 255,
    validation_split=val_split,
)

train_generator = train_datagen.flow_from_directory(
    directory=frame_train_data_dir,
    class_mode="binary",
    subset="training",
    batch_size=batch_size,
    color_mode="grayscale",
    target_size=target_size,
    classes=["Nonleaks", "Leak"]

)

val_generator = train_datagen.flow_from_directory(
    directory=frame_train_data_dir,
    class_mode="binary",
    subset="validation",
    batch_size=batch_size,
    color_mode="grayscale",
    target_size=target_size,
    classes=["Nonleaks", "Leak"]

)


test_datagen = ImageDataGenerator(
    # featurewise_center=True,
    # featurewise_std_normalization=True,
    rescale=1. / 255,
)

test_generator = test_datagen.flow_from_directory(
    directory=frame_test_data_dir, 
    class_mode='binary', 
    batch_size=batch_size,
    color_mode="grayscale",
    target_size=target_size,
    classes=["Nonleaks", "Leak"]

)


Found 250316 images belonging to 2 classes.
Found 62577 images belonging to 2 classes.
Found 208668 images belonging to 2 classes.


In [14]:
total_train_frames = total_train_NonLeak + total_train_Leak
total_train_frames

312893

In [15]:
total_batches = train_generator.__len__() + val_generator.__len__()
total_batches

9779

In [16]:
train_generator.class_indices

{'Nonleaks': 0, 'Leak': 1}

## Class Weights

In [17]:
nonleaks = total_train_NonLeak
leaks = total_train_Leak
total = nonleaks + leaks

weight_nonleak = (1 / nonleaks) * (total / 2.0)
weight_leak = (1 / leaks) * (total / 2.0)

class_weight = {train_generator.class_indices["Nonleaks"]: weight_nonleak, train_generator.class_indices["Leak"]: weight_leak}
class_weight

{0: 4.372578887056654, 1: 0.564556464126677}

In [18]:
def generator_mean(generators):
    feature_sum = 0
    num_elements = 0
    total_batches = 0
    for generator in generators: #had to include this as I am splitting training data with validation data
        batches = 0
        print("Length of Generator", len(generator))
        for data, _ in generator:
            feature_sum += np.sum(data, axis=(0, 1, 2), dtype=np.float64)
            num_elements += np.prod(data.shape[0:3], dtype=np.int64)
            if batches >= len(generator):
                # we need to break the loop by hand because
                # the generator loops indefinitely
                print("Done with one generator!")
                break
            batches += 1
            total_batches += 1
    print("Total Batches", total_batches)
    for generator in generators:
        generator.reset()
        print("Reset Generator")
    return feature_sum/num_elements

def generator_std(generators, mean, centerFirst):
    sum_squared_diff = 0
    num_elements = 0
    total_batches = 0
    og_gens = generators
    lengths_of_gens = [len(gen) for gen in generators]
    if centerFirst:
        generators = [get_with_featurewise_center(gen, mean) for gen in generators]
    for i, generator in enumerate(generators): 
        batches = 0
        print("Length of Generator", lengths_of_gens[i])
        for data, _ in generator:
            squared_diff = (data - mean) ** 2
            sum_squared_diff += np.sum(squared_diff, axis=(0, 1, 2), dtype=np.float64)
            num_elements += np.prod(data.shape[0:3], dtype=np.int64)

            if batches >= lengths_of_gens[i]:
                # we need to break the loop by hand because
                # the generator loops indefinitely
                print("Done with one generator!")
                break
            batches += 1
            total_batches += 1
    print("Total Batches", total_batches)
    for generator in og_gens:
        generator.reset()
        print("Reset Generator")
    return np.sqrt(sum_squared_diff / (num_elements - 1), dtype=np.float64)

def get_with_featurewise_center(generator, mean):
    for data, labels in generator:
        if mean:
            data -= mean
        yield data, labels

def get_with_featurewise_std_norm(generator, std):
    for data, labels in generator:
        if std:
            data /= (std + 1e-6)
        yield data, labels

def get_with_featurewise_center_std_norm (generator, mean, std):
    for data, labels in generator:
        if mean:
            data -= mean
        if std:
            data /= (std + 1e-6)
        yield data, labels

def generate_stats(gens, centerFirst):
    print("Calculating mean...")
    mean = generator_mean(generators=gens)
    print("Done!")
    print("Calculating std from centered data...")
    std = generator_std(generators=gens, mean=mean, centerFirst=centerFirst)
    print("Done!")
    return mean, std

In [19]:
mean, std = generate_stats(gens=[train_generator, val_generator], centerFirst=True)
mean, std #takes a long time not sure if there is an infinite loop - correction there was an infinite loop :))) now should run for less than 10 mins 

Calculating mean...
Length of Generator 7823
Done with one generator!
Length of Generator 1956
Done with one generator!
Total Batches 9779
Reset Generator
Reset Generator
Done!
Calculating std from centered data...
Length of Generator 7823
Done with one generator!
Length of Generator 1956
Done with one generator!
Total Batches 9779
Reset Generator
Reset Generator
Done!


(array([0.00709406]), array([0.01582667]))

In [20]:
mean = mean[0]
std = std[0]
mean, std

(0.007094055845757954, 0.01582666974503869)

## Sample Weigths

In [14]:
#TODO: calculate sample_weights using train_generator | should be similar to how predictions are made at the bottom of the notebook
#I think this should work but not sure

# A DirectoryIterator yielding tuples of (x, y) where x is a numpy array 
# containing a batch of images with shape (batch_size, *target_size, channels) 
# and y is a numpy array of corresponding labels. (In our case its train_generator)

# def sample_weights(data):
#     weights = []
#     zero_sum = 0
#     for data, _ in train_generator:
#        batch_size = data.shape[0]
#        for image_index in range(batch_size):
#             image = data[image_index]
#             summed_pixels = np.sum(image)
#             if summed_pixels == 0:
#                 weights.append(0)
#                 zero_sum += 1
#             else:
#                 # try using sqrt transformation for weight skew
#                 # weights.append(1 / np.sqrt(summed_pixels))
#                 weights.append(1 / summed_pixels)
#     train_generator.reset()
#     median_weight = np.median(weights)
#     weights = [median_weight if weight == 0 else weight for weight in weights]
#     return weights

## Model

In [21]:
from keras import layers 
from keras import models 

# model = models.Sequential() 

# # Conv Pool 1
# model.add(layers.Conv2D(4, (3, 3), input_shape=(240, 320, 1)))
# model.add(layers.BatchNormalization())
# model.add(layers.ReLU())
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.5))

# # Conv Pool 2
# model.add(layers.Conv2D(8, (3, 3)))
# model.add(layers.BatchNormalization())
# model.add(layers.ReLU())
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.5))

# # Conv Pool 3
# model.add(layers.Conv2D(8, (3, 3)))
# model.add(layers.BatchNormalization())
# model.add(layers.ReLU())
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.5))

# # Conv Pool4
# model.add(layers.Conv2D(4, (3, 3)))
# model.add(layers.BatchNormalization())
# model.add(layers.ReLU())
# model.add(layers.MaxPooling2D((2, 2)))
# model.add(layers.Dropout(0.5))

# model.add(layers.Flatten())

# model.add(layers.Dense(2400, activation='relu')) 
# model.add(layers.Dropout(0.5))
# model.add(layers.Dense(32, activation='relu')) 
# model.add(layers.Dense(1, activation='sigmoid')) 

# model.summary()


model = models.Sequential() 

# Conv Pool 1
model.add(layers.Conv2D(4, (3, 3), input_shape=(240, 320, 1)))
model.add(layers.ReLU())
# model.add(layers.Dropout(0.5))
# Remove dropout or use a small value, e.g., 0.2
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Conv Pool 2
model.add(layers.Conv2D(8, (3, 3)))
model.add(layers.ReLU())
# model.add(layers.Dropout(0.5))
# Consider using dropout of 0.3 here
model.add(layers.Dropout(0.3))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

# Conv Pool 3
model.add(layers.Conv2D(16, (3, 3)))
model.add(layers.ReLU())
# model.add(layers.Dropout(0.5))
# Consider using dropout of 0.4 here
model.add(layers.Dropout(0.4))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())


# Conv Pool4
# testing
model.add(layers.Conv2D(32, (3, 3)))
model.add(layers.ReLU())
model.add(layers.Dropout(0.5))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.BatchNormalization())

model.add(layers.Flatten())

model.add(layers.Dense(2400, activation='relu')) 
model.add(layers.Dropout(0.5))
model.add(layers.Dense(32, activation='relu')) 
# Consider adding a dropout layer here if necessary
model.add(layers.Dropout(0.5))
model.add(layers.Dense(1, activation='sigmoid')) 

model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 238, 318, 4)       40        
                                                                 
 re_lu (ReLU)                (None, 238, 318, 4)       0         
                                                                 
 dropout (Dropout)           (None, 238, 318, 4)       0         
                                                                 
 max_pooling2d (MaxPooling2  (None, 119, 159, 4)       0         
 D)                                                              
                                                                 
 batch_normalization (Batch  (None, 119, 159, 4)       16        
 Normalization)                                                  
                                                                 
 conv2d_1 (Conv2D)           (None, 117, 157, 8)       2

## Data Visualization (Confusion Matrix and ROC Curves)

In [34]:
#code from https://neptune.ai/blog/keras-metrics
#to use to plot confusion matrix and roc curve after each epock
#uncomment when you need it 

import os

from keras.callbacks import Callback
from sklearn.metrics import RocCurveDisplay, roc_curve
from sklearn.metrics import ConfusionMatrixDisplay, confusion_matrix

class PerformanceVisualizationCallback(Callback):
    def __init__(self, model, validation_data, num_batches, image_dir):
        super().__init__()
        self.model = model
        self.validation_data = validation_data
        self.num_batches = num_batches

        os.makedirs(image_dir, exist_ok=True)
        self.image_dir = image_dir

    def on_epoch_end(self, epoch, logs={}):
        y_true = []
        y_pred = []
        batch_num = 0
        for data, true_label in self.validation_data:
            batch_pred = self.model.predict(data, verbose=0)
            y_pred.append(batch_pred)
            y_true.append(true_label)

            if batch_num >= self.num_batches:
                break
            batch_num += 1

        y_true = np.concatenate(y_true, axis=0)
        y_pred = np.concatenate(y_pred, axis=0)

        threshold = 0.5
        y_pred = (y_pred > threshold).astype(int)
        

        # plot and save confusion matrix
        fig, ax = plt.subplots(figsize=(16,12))
        cm = confusion_matrix(y_true, y_pred)
        # cm_display = ConfusionMatrixDisplay(cm).plot(ax=ax, name=f"Confusion Matrix of Validation Data at Epoch {epoch}")
        cm_display = ConfusionMatrixDisplay(cm).plot(ax=ax)
        ax.set_title(f"Confusion Matrix of Validation Data at Epoch {epoch}")
        fig.savefig(os.path.join(self.image_dir, f'confusion_matrix_epoch_{epoch}'))

        # plot and save roc curve
        fig, ax = plt.subplots(figsize=(16,12))
        
        fpr, tpr, _ = roc_curve(y_true, y_pred)
        # roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax, name=f"ROC Curve of Validation Data at Epoch {epoch}")
        roc_display = RocCurveDisplay(fpr=fpr, tpr=tpr).plot(ax=ax)
        ax.set_title(f"ROC Curve of Validation Data at Epoch {epoch}")
        fig.savefig(os.path.join(self.image_dir, f'roc_curve_epoch_{epoch}'))

performance_cbk = PerformanceVisualizationCallback(
                      model=model,
                      validation_data=get_with_featurewise_center_std_norm(val_generator, mean, std),
                      num_batches=len(val_generator),
                      image_dir='performance_vizualizations')

## Training Model

In [35]:
from keras import optimizers 
from keras import metrics
from keras.callbacks import EarlyStopping, ModelCheckpoint

def F1Score(y_true, y_pred):
    threshold = 0.5
    y_pred = tf.where(y_pred >= threshold, 1.0, 0.0)

    prec = tf.keras.metrics.Precision()
    recall = tf.keras.metrics.Recall()
    
    prec.update_state(y_true, y_pred)
    recall.update_state(y_true, y_pred)
    
    prec_res = prec.result()
    rec_res = recall.result()

    return tf.where(tf.math.add(prec_res, rec_res) > 0, 
                    2 * (prec_res * rec_res) / (prec_res + rec_res), 
                    0.0)


model.compile(loss='binary_crossentropy', optimizer=optimizers.legacy.Adam(learning_rate=1e-4), metrics=[F1Score, "acc"], run_eagerly=True)


In [38]:
early_stop = EarlyStopping(monitor = 'val_loss', patience = 3, restore_best_weights = True)


num_epochs = 10

history = model.fit(
    x = get_with_featurewise_center_std_norm(train_generator, mean, std),
    steps_per_epoch = train_generator.samples // batch_size,
    validation_data = get_with_featurewise_center_std_norm(val_generator, mean, std),
    validation_steps = val_generator.samples // batch_size,
    epochs = num_epochs,
    class_weight = class_weight,
    callbacks=[performance_cbk, early_stop] #uncomment once you want to use it
)

Epoch 1/10
7822/7822 [==============================] - ETA: 0s - loss: 0.0087 - F1Score: 0.9986 - acc: 0.9976

7822/7822 [==============================] - 3030s 387ms/step - loss: 0.0087 - F1Score: 0.9986 - acc: 0.9976 - val_loss: 8.5570 - val_F1Score: 0.0000e+00 - val_acc: 0.1143
Epoch 2/10
7822/7822 [==============================] - ETA: 0s - loss: 0.0052 - F1Score: 0.9991 - acc: 0.9984

7822/7822 [==============================] - 2907s 372ms/step - loss: 0.0052 - F1Score: 0.9991 - acc: 0.9984 - val_loss: 5.6521 - val_F1Score: 0.0000e+00 - val_acc: 0.1143
Epoch 3/10
1076/7822 [===>..........................] - ETA: 37:43 - loss: 0.0033 - F1Score: 0.9994 - acc: 0.9990

## Model Metrics Over Epochs

In [ ]:
import matplotlib.pyplot as plt 

f1 = history.history['F1Score'] 
val_f1 = history.history['val_F1Score'] 

loss = history.history['loss'] 
val_loss = history.history['val_loss'] 

acc = history.history["acc"]
val_acc = history.history["val_acc"]

epochs = range(1, len(f1) + 1) 

plt.plot(epochs, f1, 'bo', label='Training F1 Score') 
plt.plot(epochs, val_f1, 'b', label='Validation F1 Score') 
plt.title('Training and Validation F1 Score') 
plt.legend() 

plt.figure() 

plt.plot(epochs, loss, 'bo', label='Training loss') 
plt.plot(epochs, val_loss, 'b', label='Validaion loss') 
plt.title('Training loss and validation loss') 
plt.legend() 

plt.plot(epochs, acc, 'bo', label='Training Accuracy') 
plt.plot(epochs, val_acc, 'b', label='Validation Accuracy') 
plt.title('Training and Validation Accuracy') 
plt.legend() 

plt.figure() 

plt.show() 

## Testing

In [ ]:
test_mean, test_std = generate_stats(gens=[test_generator], centerFirst=True)
test_mean, test_std

In [ ]:
test_mean = test_mean[0]
test_std = test_std[0]

In [ ]:
#source: https://stackoverflow.com/questions/45413712/keras-get-true-labels-y-test-from-imagedatagenerator-or-predict-generator

# Create lists for storing the predictions and labels
predictions = []
true_labels = []

threshold = 0.5
# Get the total number of labels in generator 
# (i.e. the length of the dataset where the generator generates batches from)
length_test = len(test_generator.labels)

# Loop over the generator
for data, label in get_with_featurewise_center_std_norm(test_generator, test_mean, test_std):
    # Make predictions on data using the model. Store the results.
    preds = model.predict(data, verbose=0)
    processed_preds = (preds >= threshold).flatten().astype(int)
    predictions.extend(processed_preds)
    # Store corresponding labels
    true_labels.extend(label.astype(int))
    
    # We have to break out from the generator when we've processed 
    # the entire once (otherwise we would end up with duplicates). 
    if len(predictions) == length_test:
        break

In [ ]:
len(predictions), len(true_labels)

In [ ]:
from sklearn.metrics import confusion_matrix

#source: https://vitalflux.com/python-draw-confusion-matrix-matplotlib/

conf_matrix = confusion_matrix(y_true=true_labels, y_pred=predictions)
#
# Print the confusion matrix using Matplotlib
#
fig, ax = plt.subplots(figsize=(7.5, 7.5))
ax.matshow(conf_matrix, cmap=plt.cm.Blues, alpha=0.3)
for i in range(conf_matrix.shape[0]):
    for j in range(conf_matrix.shape[1]):
        ax.text(x=j, y=i,s=conf_matrix[i, j], va='center', ha='center', size='xx-large')
 
plt.xlabel('Predictions', fontsize=18)
plt.ylabel('Actuals', fontsize=18)
plt.title('Confusion Matrix (0: Nonleak, 1:Leak)', fontsize=18)
plt.show()

In [ ]:
# Extract the true positive, true negative, false positive, and false negative values from the confusion matrix
tn, fp, fn, tp = conf_matrix.ravel()
 
# Print the true positive, true negative, false positive, and false negative values
print("True Positive (TP): ", tp)
print("True Negative (TN): ", tn)
print("False Positive (FP): ", fp)
print("False Negative (FN): ", fn)
 
# Calculate accuracy
accuracy = (tp + tn) / (tp + tn + fp + fn)
 
# Calculate precision
precision = tp / (tp + fp)
 
# Calculate recall
recall = tp / (tp + fn)
 
# Calculate F1-score
f1_score = 2 * (precision * recall) / (precision + recall)
 
# Print the formulas for accuracy, precision, recall, and F1-score
print("\n\nFormulas:")
print("Accuracy: (TP + TN) / (TP + TN + FP + FN)")
print("Precision: TP / (TP + FP)")
print("Recall: TP / (TP + FN)")
print("F1-score: 2 * (Precision * Recall) / (Precision + Recall)")
 
# Print the accuracy, precision, recall, and F1-score
print("\n\nMetrics:")
print("Accuracy: ", round(accuracy, 4))
print("Precision: ", round(precision, 4))
print("Recall: ", round(recall, 4))
print("F1-score: ", round(f1_score, 4))

In [ ]:


def get_test_FN_frames(generator, total):
    res = []
    fn_indices = [i for i in range(len(predictions)) if predictions[i] == 0 and true_labels[i] == 1]
    fn_indices = np.random.choice(fn_indices, total, replace=False)
    fn_set = set(fn_indices)
    batch_num = 0
    for i in range(total):
        i_fn = fn_indices[i]
        i_batch = i_fn // batch_size
        data, _ = generator.__getitem__(i_batch)
        res.append(data[i_fn % batch_size])
    return res
frames_FN = get_test_FN_frames(test_generator, 10)
frames_FN

In [ ]:
def plot_images(images):
    for i, img in enumerate(images):
        cv2.imshow(f"Image {i + 1}", img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
plot_images(frames_FN)